In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States')
states = [s.replace('[E]', '').replace(' ', '-') for s in list(
    dfs[0]['Flag, name andpostal abbreviation[12]']['Flag, name andpostal abbreviation[12]']
)]
', '.join(states)

In [4]:
def load_state(state):
    print(f'loading data for state {state}')
    page = pd.read_html(f'https://www.nytimes.com/interactive/2020/11/03/us/elections/results-{state}.html')
    df = page[1]
    df = pd.DataFrame(df.iloc[:-1]) # last row is 'view results'
    df['Total votes'] = df['Total votes'].astype(int)
    df['State'] = state
    return df.sort_values('Total votes', ascending = False)

In [44]:
try:
    us_df = pd.read_pickle('./2020_election_data.pickle')
except:
    result = {}
    for s in states:
        s = s.lower()
        result[s] = load_state(s)
    
    us_df = pd.concat(result.values())
    us_df.to_pickle('./2020_election_data.pickle')
finally:
    print(f'Total: {len(us_df)}')
    display(us_df.head())

Total: 4623


,County,Margin,2016 margin,Est. votes reported,Est. votes reported.1,Total votes,Absentee,State,District,Town,Parish
4,Jefferson,Biden +13,D+7.3,>98%,NaN,324675,—,alabama,NaN,NaN,NaN
5,Madison,Trump +8,R+16.3,>98%,NaN,193955,—,alabama,NaN,NaN,NaN
0,Mobile,Trump +12,R+13.3,93%,NaN,181783,—,alabama,NaN,NaN,NaN
6,Shelby,Trump +41,R+49.4,>98%,NaN,114439,—,alabama,NaN,NaN,NaN
7,Baldwin,Trump +54,R+57.2,>98%,NaN,108945,—,alabama,NaN,NaN,NaN


In [45]:
# assuming other parties are very minimal and won't affect the votes estimation
us_df['2020_Who'] = np.where(us_df['Margin'].str.split('+', expand=True)[0].str.strip() == 'Biden','D','R')
us_df['2020_Margin'] = pd.to_numeric(us_df['Margin'].str.split('+', expand=True)[1])
us_df['2016_Who'] = us_df['2016 margin'].str.split('+', expand=True)[0].str.strip()
us_df['2016_Margin'] = pd.to_numeric(us_df['2016 margin'].str.split('+', expand=True)[1])
us_df.head()

,County,Margin,2016 margin,Est. votes reported,Est. votes reported.1,Total votes,Absentee,State,District,Town,Parish,2020_Who,2020_Margin,2016_Who,2016_Margin
4,Jefferson,Biden +13,D+7.3,>98%,NaN,324675,—,alabama,NaN,NaN,NaN,D,13.0,D,7.3
5,Madison,Trump +8,R+16.3,>98%,NaN,193955,—,alabama,NaN,NaN,NaN,R,8.0,R,16.3
0,Mobile,Trump +12,R+13.3,93%,NaN,181783,—,alabama,NaN,NaN,NaN,R,12.0,R,13.3
6,Shelby,Trump +41,R+49.4,>98%,NaN,114439,—,alabama,NaN,NaN,NaN,R,41.0,R,49.4
7,Baldwin,Trump +54,R+57.2,>98%,NaN,108945,—,alabama,NaN,NaN,NaN,R,54.0,R,57.2


In [47]:
us_df[us_df['2016_Who'].isnull()].State.unique()

array(['alabama', 'arizona', 'arkansas', 'california', 'colorado',
       'delaware', 'florida', 'georgia', 'hawaii', 'idaho', 'illinois',
       'indiana', 'iowa', 'kansas', 'kentucky', 'louisiana', 'maryland',
       'michigan', 'minnesota', 'mississippi', 'missouri', 'montana',
       'nebraska', 'nevada', 'new-jersey', 'new-mexico', 'new-york',
       'north-carolina', 'north-dakota', 'ohio', 'oklahoma', 'oregon',
       'pennsylvania', 'south-carolina', 'south-dakota', 'tennessee',
       'texas', 'utah', 'virginia', 'washington', 'west-virginia',
       'wisconsin', 'wyoming'], dtype=object)

In [74]:
vote_count_threshold = 10000
has_2016_data = us_df[~us_df['2016_Who'].isnull()]
shifted = has_2016_data.loc[lambda x: x['2020_Who'] != x['2016_Who']].loc[lambda x: x['Total votes'] > vote_count_threshold]
#display(shifted)
print("% of the county shifted to a different party : " + str(len(shifted) / len(has_2016_data)))

def print_q(df):
    print('Margin Percentile: ')
    print((df['2020_Margin'] + df['2016_Margin']).quantile([0.25, 0.5, 0.75, 0.9, 0.95, 0.99]))

to_d = shifted.loc[lambda x: x['2020_Who'] == 'D']
to_r = shifted.loc[lambda x: x['2020_Who'] == 'R']

def against_df(msg, df):
    print('------')
    print(msg)
    print('------')
    print(f'len : {len(df)}')
    print_q(df)
    print(f'States: {df.State.unique()}')
    
print(f'From R to D: {len(to_d)}, From D to R: {len(to_r)}')

against_df('From R to D', to_d)
against_df('From D to R', to_r)

print('States that only move to D:')
display(set(to_d.State.unique()) - set(to_r.State.unique()))
print('States that only move to R:')
display(set(to_r.State.unique()) - set(to_d.State.unique()))

% of the county shifted to a different party : 0.020374630299046993
From R to D: 46, From D to R: 16
------
From R to D
------
len : 46
Margin Percentile: 
0.25     4.625
0.50     6.045
0.75     9.075
0.90    11.400
0.95    11.725
0.99    14.915
dtype: float64
States: ['arizona' 'california' 'colorado' 'florida' 'illinois' 'indiana' 'kansas'
 'maryland' 'michigan' 'minnesota' 'mississippi' 'new-jersey'
 'north-carolina' 'ohio' 'oregon' 'pennsylvania' 'texas' 'virginia'
 'washington' 'wisconsin' 'wyoming']
------
From D to R
------
len : 16
Margin Percentile: 
0.25     3.040
0.50     6.050
0.75     9.975
0.90    17.550
0.95    18.500
0.99    19.940
dtype: float64
States: ['georgia' 'illinois' 'mississippi' 'new-york' 'north-carolina' 'ohio'
 'south-carolina' 'texas']
States that only move to D:


{'arizona',
 'california',
 'colorado',
 'florida',
 'indiana',
 'kansas',
 'maryland',
 'michigan',
 'minnesota',
 'new-jersey',
 'oregon',
 'pennsylvania',
 'virginia',
 'washington',
 'wisconsin',
 'wyoming'}

States that only move to R:


{'georgia', 'new-york', 'south-carolina'}

In [53]:
has_2016_data[['2020_Margin', '2016_Margin']].corr()

,2020_Margin,2016_Margin
2020_Margin,1.000000,0.969486
2016_Margin,0.969486,1.000000


In [13]:
# ignore small counties
us_df.loc[lambda x: x['Total votes'] > 5000].corr()

,Total votes,2020_Margin,2016_Margin
Total votes,1.000000,-0.125740,-0.167907
2020_Margin,-0.125740,1.000000,0.965651
2016_Margin,-0.167907,0.965651,1.000000
